# Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa. 
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt** 
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [107]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np


np.seterr(divide = 'ignore')
iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [108]:
class NaiveBayes:
    def __init__(self):
        self.priors = {}
        # entries in likelihood are matrices, where rows are 4-long arrays of likelihoods of each bin
        self.likelihoods = {}
        self.binCount = 4
        self.bins = {}

    def build_classifier(self, train_features, train_classes):

        # likelihoods - probability that a feature has given value when we know class is `i`.
        # Calculated from normal distribution
        normalise = lambda v : v/len(train_classes)
        counts = Counter(train_classes)
        self.priors = {k: normalise(v) for k, v in counts.items()}


        discrete_features = self.data_discretization(train_features, self.binCount)

        # for every class
        for c in counts.keys():
            # get rows with matching class
            matching = np.array([row for idx,row in enumerate(discrete_features) if train_classes[idx]==c])
            # find ratio between all and found
            normalise = lambda v : v/len(matching)
            x = {}
            # do this for all columns
            for i,col in enumerate(matching.T):
                # count how many times this feature appeared
                featureCounts = Counter(col)
                x[i] = {k: normalise(v) for k, v in featureCounts.items()}
                # add zeros if feature didnt show up
                for bin in range(1,self.binCount+1):
                    if bin not in col:
                        x[i][bin]=0
            
            self.likelihoods[c]=x # for this class, here's a likehood dict


        return self

    def calculateBins(self, whole_data):
        for i,col in enumerate(whole_data.T):
            cmin,cmax = col.min(),col.max()
            self.bins[i] = np.linspace(cmin, cmax, self.binCount+1)
        return self

    #@staticmethod
    def data_discretization(self,data, segment_count):
        digitized_arr = np.zeros_like(data,dtype=np.int8)
        for i in range(data.shape[1]):
            column = data[:, i]
            digitized_arr[:, i] = np.digitize(column, self.bins[i], right=True)
        return digitized_arr


    def predict(self, sample):
        # calculate probability of each class, return most probable one
        # the idea is that we assume that features are independent, so 
        # the likehood of given features vector in some class is
        # product of likehoods of singular features 
        predictions = {}
        discreteSample = [np.digitize(feature, self.bins[i]) for i,feature in enumerate(sample)]
        for cls in self.priors:
            #P(cechy|cls)=PI(cecha_i|cls)
            likelihood=1
            for i,value in enumerate(discreteSample):
                likelihood*=self.likelihoods[cls][i][value]
            predictions[cls]=likelihood*self.priors[cls]
        
        return max(predictions, key=predictions.get) 


class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        #self.likelihoods = {}
        self.means = {}
        self.stds = {}

    def build_classifier(self, train_features, train_classes):
        # priors - probability of class in whole dataset
        # priors[i] = p(klasa=i) dla i € klasy
        normalise = lambda v : v/len(train_classes)
        counts = Counter(train_classes)
        self.priors = {k: normalise(v) for k, v in counts.items()}

        # calculate mean and std of each feature for each class
        # then likelihood is return of normal distribution density with these params at given point
        for c in counts.keys():
            matching = np.array([row for idx,row in enumerate(train_features) if train_classes[idx]==c])
            self.means[c]=np.array([np.mean(column) for column in matching.T])
            self.stds[c]=np.array([np.std(column) for column in matching.T])

        return self

    @staticmethod
    def normal_dist(x: np.ndarray, mean: np.ndarray, std: np.ndarray):
        return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.power((x - mean)/std, 2)/2)

    def predict(self, sample):
        # like previous, but now we estimate likehoods of the features based on
        # mean and std
        #           ->      ->                      -> 
        # P(klasa|cechy)=P(cechy|klasa)*P(klasa)/P(cechy)
        #   * dla każdej klasy liczymy P, wybieramy klase z max wynikiem
        #   * P(cechy) ma jakąś wartośc, ale dla wybrania maksymalnego wyniku można pominąć,
        #     bo i tak wyniki będą proporcjonalne             -> 
        #   * P(cechy|klasa) liczymy właśnie z gaussa, ale P(cechy|klasa)=P(cecha1|klasa)*P(cecha2|klasa)*...
        #     bo zakładamy że niezależne
        predictions = {}
        for cls in self.priors:
            likelihood = np.prod(self.normal_dist(sample,self.means[cls],self.stds[cls]))
            predictions[cls]=likelihood*self.priors[cls]
        
        return max(predictions, key=predictions.get) 





## Seed 123

In [109]:

normal = NaiveBayes().calculateBins(x).build_classifier(x_train,y_train)
gaussian = GaussianNaiveBayes().build_classifier(x_train,y_train)

gaussianHits,normalHits = 0,0

for x,y in zip(x_test,y_test):
    gaussianHits+=gaussian.predict(x)==y
    normalHits+=normal.predict(x)==y

print(f"Gaussian accuracy: {gaussianHits}/{len(y_test)} {100*gaussianHits/len(y_test)}%")
print(f"Normal accuracy: {normalHits}/{len(y_test)} {100*normalHits/len(y_test)}%")



Gaussian accuracy: 15/15 100.0%
Normal accuracy: 14/15 93.33333333333333%


In [112]:
def RunAverageTests(testCount,datax,datay):
    for split in (0.1,0.3,0.5,0.7):
        gaussianHits,normalHits = 0,0
        for i in range(testCount):
            x_train, x_test, y_train, y_test = train_test_split(datax, datay, test_size=split)
            normal = NaiveBayes().calculateBins(datax).build_classifier(x_train,y_train)
            gaussian = GaussianNaiveBayes().build_classifier(x_train,y_train)

            for x,y in zip(x_test,y_test):
                #print(gaussian.predict(x),y)
                gaussianHits+=gaussian.predict(x)==y
                normalHits+=normal.predict(x)==y
        print("split:",split)
        print(f"Gaussian accuracy: {gaussianHits}/{len(y_test)*testCount} {100*gaussianHits/(len(y_test)*testCount)}%")
        print(f"Normal accuracy: {normalHits}/{len(y_test)*testCount} {100*normalHits/(len(y_test)*testCount)}%")
    

iris = load_iris()
RunAverageTests(1000,iris.data,iris.target)

split: 0.1
Gaussian accuracy: 14287/15000 95.24666666666667%
Normal accuracy: 12638/15000 84.25333333333333%
split: 0.3
Gaussian accuracy: 42886/45000 95.30222222222223%
Normal accuracy: 38164/45000 84.80888888888889%
split: 0.5
Gaussian accuracy: 71382/75000 95.176%
Normal accuracy: 63467/75000 84.62266666666666%
split: 0.7
Gaussian accuracy: 99469/105000 94.73238095238095%
Normal accuracy: 87285/105000 83.12857142857143%


# Wnioski

Klasyfikator "niezależny" nosi swoją nazwę z powodu, że zakładamy niezależność zmiennych (cech), przez co policzenie prawdopodobnści oznacza policzenie iloczynu pojedynczych prawdopodobności.


Wyniki dla `random_seed=123` są 15/15 dla Gaussa oraz 14/15 dla dyskretyzatora:
```
Gaussian accuracy: 15/15 100.0%
Normal accuracy: 14/15 93.33333333333333%
```

Średnia celnośc obydwu modeli z 1000 uruchomień::
* podział zbioru 0.1
    * Gaussian: 14280/15000 95.2%
    * Dyskretyzator: 12638/15000 84.25%
* podział zbioru 0.3
    * Gaussian: 42932/45000 95.40%
    * Dyskretyzator: 38164/45000 84.80%
* podział zbioru 0.5
    * Gaussian: 71452/75000 95.27%
    * Dyskretyzator: 63467/75000 84.62%
* podział zboiru 0.7
    * Gaussian: 99367/105000 94.64%
    * Dyskretyzator: 87285/105000 83.12%


Podejście z dyskretyzowaniem zmiennych na 4 "kubełki" wypada trochę gorzej od gaussa.
